In [19]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

import shap

In [20]:
wine_qt_path = '../data/WineQT.csv'
RANDOM_SEED = 492

In [21]:
wine_df = pd.read_csv(wine_qt_path)

In [22]:
print(wine_df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  Id  
0      9.4        5   0  
1      9.8        5   1  
2      9

In [23]:
y_series = wine_df['Id']
y = pd.DataFrame(y_series, columns=['Id'])
features = [col for col in wine_df.columns if col != 'Id']
X = wine_df[features]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [25]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [26]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
print(X_train_scaled.head())

In [28]:
subset_indices = np.random.choice(X_train.index, size=len(X_train), replace=False)

X_train_subset = X_train.loc[subset_indices]
y_train_subset = y_train.loc[subset_indices]

print(X_train_scaled.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       0.105544         -1.379153     1.462888       -0.537610   0.087465   
1       0.621720         -1.046416     0.500012       -0.244623  -0.252294   
2       0.334956         -1.379153     0.905434       -0.610857  -0.412180   
3      -0.640043          0.229075    -1.121673       -0.171377  -0.132379   
4      -0.869454          0.173619    -1.223028       -0.610857  -0.072421   

   free sulfur dioxide  total sulfur dioxide   density        pH  sulphates  \
0             1.921184              1.727330  0.097116 -0.055335   0.537430   
1            -1.030261             -0.990201  0.667681 -0.564916  -0.225502   
2            -0.833498             -1.020395 -1.645700 -0.373823  -0.636312   
3             0.051936             -0.295721 -0.556440  0.454246  -0.342877   
4             0.248699             -0.325915  0.045247  0.836432  -0.166815   

    alcohol   quality  
0  0.042265  1.653380  
1 -1.146

In [29]:
X_train_subset = X_train_subset.reset_index(drop=True)
y_train_subset = y_train_subset.reset_index(drop=True)

In [30]:
X_train_tensor = tf.constant(X_train_subset.values, dtype=tf.float32)
y_train_tensor = tf.constant(y_train_subset.values, dtype=tf.float32)
print(y_train_tensor.shape)

(914, 1)


In [35]:
class GradientBoostedTreesEnsembleClassifier(tf.keras.Model):
    def __init__(self, n_trees=100, max_depth=3, n_estimators=100, subportion = 0.8, sample_method='RANDOM'): 
        super(GradientBoostedTreesEnsembleClassifier, self).__init__()
        self.n_trees = n_trees
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subportion = subportion
        self.sample_method = sample_method
        self.estimators = []

    def build_estimator(self):
        model = tfdf.keras.GradientBoostedTreesModel(
            task=tfdf.keras.Task.CLASSIFICATION,
            num_trees=np.random.randint(self.n_trees/5, self.n_trees+1), # self.n_trees, #
            max_depth=self.max_depth, #np.random.randint(self.max_depth/2, self.max_depth+1), # self.max_depth, # 
            validation_ratio=0.1,
            subsample=self.subportion,
            sampling_method=self.sample_method,
            random_seed=self.seed # try to set this first. Maybe this is enough
        )
        return model
    
    # if not, the number of trees/depth different for each estimator

    def fit(self, X, y):
        
        for seed in np.random.randint(0, 1000000, size=self.n_estimators):
            self.seed = int(seed)
            subset_size = int(self.subportion * len(X))
            
            # Create a subset of the dataset using sampled indices
            subset_indices = np.random.choice(X.index, size=subset_size, replace=False)
            X_train = X.loc[subset_indices].reset_index(drop=True)
            y_train = y.loc[subset_indices].reset_index(drop=True)
            
            # X_tensor = tf.constant(X_train.values, dtype=tf.float32)
            # y_tensor = tf.constant(y_train.values, dtype=tf.float32)

            # subset_dataset = (X_tensor, y_tensor)
            
            # building each estimator
            estimator = self.build_estimator()
            estimator.fit(X_train,y_train,verbose=0)
            self.estimators.append(estimator)
    
    def predict_mean(self, X): # use all estimators
        predictions = []
        
        for i, estimator in enumerate(self.estimators):
            estimator_predictions = estimator.predict(X, verbose=0)  # Get predictions from the current estimator
            print(f"Shape of prediction from estimator {i}: {estimator_predictions.shape}")  # Print the shape
            predictions.append(estimator_predictions)
        
        predictions = [pred if len(pred.shape) == 2 else pred[:, np.newaxis] for pred in predictions]

        # Stack predictions for mean and standard deviation calculation
        predictions_stacked = np.stack(predictions, axis=-1)
        
        mean_prediction = np.mean(predictions_stacked, axis=-1)
        sd_prediction = np.std(predictions_stacked, axis=-1)
        return mean_prediction, sd_prediction


In [36]:
gbt_model = GradientBoostedTreesEnsembleClassifier(
    n_trees=50, 
    max_depth=5, 
    n_estimators=80, 
    subportion=0.8, 
    sample_method='RANDOM'
)

In [ ]:
gbt_model.fit(X_train_scaled, y_train)

Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpz7kb8pwy as temporary training directory


[WARNING 24-06-05 17:24:06.2575 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:24:06.2576 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:24:06.2576 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 17:24:28.4320 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpz7kb8pwy/model/ with prefix 4e7cd3d0b4dc4eaf
[INFO 24-06-05 17:24:29.0867 CEST decision_forest.cc:734] Model loaded with 17534 root(s), 314048 node(s), and 12 input feature(s).
[INFO 24-06-05 17:24:29.0868 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 17:24:29.0868 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmposye5cpw as temporary training directory


[WARNING 24-06-05 17:24:29.4242 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:24:29.4242 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:24:29.4243 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 17:24:52.8833 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmposye5cpw/model/ with prefix 9dd0df16d7524c43
[INFO 24-06-05 17:24:53.5368 CEST decision_forest.cc:734] Model loaded with 17578 root(s), 330102 node(s), and 12 input feature(s).
[INFO 24-06-05 17:24:53.5369 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 17:24:53.5369 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpp51im1_q as temporary training directory


...

[WARNING 24-06-05 17:43:32.0636 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:43:32.0636 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:43:32.0636 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 17:43:52.5245 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpcm9244vq/model/ with prefix 98ade1a2f8834c15
[INFO 24-06-05 17:43:53.1043 CEST decision_forest.cc:734] Model loaded with 17556 root(s), 333536 node(s), and 12 input feature(s).
[INFO 24-06-05 17:43:53.1043 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 17:43:53.1043 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpmxnqxyxm as temporary training directory


[WARNING 24-06-05 17:43:53.3104 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:43:53.3104 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:43:53.3104 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 17:44:13.3896 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpmxnqxyxm/model/ with prefix 069a3e15c0ed4bb0
[INFO 24-06-05 17:44:13.9515 CEST decision_forest.cc:734] Model loaded with 17578 root(s), 325730 node(s), and 12 input feature(s).
[INFO 24-06-05 17:44:13.9516 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 17:44:13.9516 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpce_o1ph8 as temporary training directory


[WARNING 24-06-05 17:44:14.2164 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:44:14.2164 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 17:44:14.2164 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


In [34]:
mean_predict, sd_predict = gbt_model.predict_mean(X_test_scaled)

2024-06-05 16:00:20.332044: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [229,1598] != values[6].shape = [229,1596]


InvalidArgumentError: {{function_node __wrapped__Pack_N_80_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [229,1598] != values[6].shape = [229,1596] [Op:Pack] name: stack

In [ ]:
y_test_eval = np.array(y_test).flatten()
y_pred_eval = mean_predict.numpy().flatten()

In [ ]:
PLOT = 30

In [ ]:
y_plot = y_test[:PLOT]
x_axis = np.arange(len(y_plot))
y_mean_predict = mean_predict[:PLOT].numpy()
y_var_predict = variance_predict[:PLOT].numpy()

In [ ]:
plt.figure(figsize=(8, 6))
plt.xlim(-1,PLOT)
plt.scatter(x_axis, y_plot, color='lightblue', label="Ground Truth")
plt.scatter(x_axis, y_mean_predict.flatten(), color='pink', alpha=0.8, label='Mean Prediction')
# plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=root_mse, fmt='none', ecolor='pink', label="Root MSE")
plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=y_var_predict.flatten(), fmt='none', color='darkred', alpha=0.8, label="Uncertainty")
plt.ylabel('prices')
plt.legend()
plt.grid(True)  # Enable grid for better visualization
plt.show()

In [ ]:
# predict gives the class number
# make a histogram of the class prediction
# entropy to measure the uncertainty